In [10]:
import os
import sys
import pandas as pd
import re
import seaborn as sns
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'utils')))
import utils as ut
import ast

In [3]:
wines_df = pd.read_csv("../src/data/processed/all_wines.csv")
wines_df

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,image
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020,Luigi Bosca,4.8,575,164.99,73.43435%,50.905894999999994%,13.619732499999998%,44.74824%,"{'black fruit': 14, 'oaky': 13, 'earthy': 5, '...","['beef', 'lamb', 'poultry', 'game (deer, venis...","['Cabernet Sauvignon', 'Malbec']","['Argentina', 'Mendoza']",Argentinian Cabernet Sauvignon - Malbec,https://images.vivino.com/thumbs/_Bf6JTwYRpSX6...
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015,Catena Zapata,4.7,295,675.00,74.09986749999999%,55.86741500000001%,14.329525%,54.65305%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017,Catena Zapata,4.7,218,580.00,74.09986749999999%,55.86741500000001%,14.329525%,54.65305%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
3,https://www.vivino.com/US/en/monteviejo-la-vio...,La Violeta,2013,Monteviejo,4.7,202,150.00,73.06289000000001%,40.9165975%,21.4814425%,36.61685%,"{'oaky': 51, 'black fruit': 49, 'earthy': 26, ...","['beef', 'lamb']",['100%Malbec'],"['Argentina', 'Mendoza', 'Uco Valley']",Argentinian Uco Valley Malbec Red,https://images.vivino.com/thumbs/FO-x9h3mQHSx9...
4,https://www.vivino.com/US/en/vina-cobos-cobos-...,Cobos Volturno,2018,Viña Cobos,4.7,198,399.99,71.05489250000001%,45.630624999999995%,24.2544575%,38.45142%,"{'oaky': 86, 'black fruit': 49, 'spices': 17, ...","['beef', 'lamb', 'poultry', 'game (deer, venis...",['100%Cabernet Sauvignon'],"['Argentina', 'Mendoza', 'Perdriel']",Argentinian Cabernet Sauvignon - Malbec,https://images.vivino.com/thumbs/D1Mf1fYnRnage...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,https://www.vivino.com/etchart-cafayate-terroi...,Cafayate Terroir de Altura Torrontés,2021,Bodegas Etchart,3.9,110,NaN,47.892224999999996%,NaN,19.515900000000002%,56.4788025%,"{'tree fruit': 3, 'red fruit': 1, 'floral': 1}","['vegetarian', 'spicy food', 'aperitif']",['Torrontés'],"['Argentina', 'Salta', 'Calchaqui Valley', 'Ca...",Argentinian Torrontés,https://images.vivino.com/thumbs/zbf30yF2Q_6j_...
2022,https://www.vivino.com/siesta-cabernet-sauvign...,Cabernet Sauvignon,2011,Siesta,3.9,108,NaN,65.158975%,54.17996%,13.757972500000001%,41.570055%,"{'oaky': 22, 'black fruit': 14, 'red fruit': 1...","['beef', 'lamb', 'poultry']",['100%Cabernet Sauvignon'],"['Argentina', 'Mendoza']",Argentinian Cabernet Sauvignon,https://images.vivino.com/thumbs/hz2-G9-7TyGMC...
2023,https://www.vivino.com/gen-del-alma-jijiji-mal...,JIJIJI Malbec - Pinot Noir,2022,Gen del Alma,3.9,105,22.66,63.11037999999999%,45.074575%,21.187649999999998%,58.78908%,"{'red fruit': 60, 'black fruit': 27, 'earthy':...","['beef', 'lamb', 'pork', 'poultry', 'mushrooms...","['Malbec', 'Pinot Noir']","['Argentina', 'Mendoza', 'Uco Valley', 'Tupung...",Argentinian Malbec Red Blend,https://images.vivino.com/thumbs/hqHY7FTaRoucR...
2024,https://www.vivino.com/terrazas-de-los-andes-h...,High Altitude Vineyards Malbec,2021,Terrazas de los Andes,3.9,103,NaN,62.6411375%,31.788999999999994%,18.47617%,37.5830825%,"{'oaky': 1756, 'black fruit': 1668, 'earthy': ...","['beef', 'lamb']",['100%Malbec'],"['Argentina', 'Mendoza', 'Lujan de Cuyo']",Argentinian Lujan de Cuyo Malbec Red,https://images.vivino.com/thumbs/xaqIkl7ERfa2e...


In [3]:
wines_df.shape

(2026, 17)

In [4]:
wines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   wine_link   2026 non-null   object 
 1   name        2023 non-null   object 
 2   year        2023 non-null   object 
 3   winery      2026 non-null   object 
 4   rating      2026 non-null   float64
 5   rating_qty  2026 non-null   object 
 6   price       1601 non-null   float64
 7   body        1974 non-null   object 
 8   tannis      1663 non-null   object 
 9   sweetness   1945 non-null   object 
 10  acidity     1974 non-null   object 
 11  notes       2026 non-null   object 
 12  pairings    2026 non-null   object 
 13  grapes      2026 non-null   object 
 14  region      2026 non-null   object 
 15  style       2021 non-null   object 
 16  image       2026 non-null   object 
dtypes: float64(2), object(15)
memory usage: 269.2+ KB


In [5]:
null_tannis = wines_df[wines_df["tannis"].isna()]
ut.save_csv(null_tannis, "/")

TypeError: save_csv() missing 1 required positional argument: 'filename'

In [4]:
#Función para convertir a decimales los objets, tomando los primeros cinco caracteres
def clean_and_convert(value):
    if isinstance(value, float):  
        return value
    return pd.to_numeric(str(value)[:5].replace('%', ''), errors='coerce')

# Aplicar la conversión a las columnas deseadas
columns_to_convert = ['body', 'tannis', 'sweetness', 'acidity']
for col in columns_to_convert:
    wines_df[col] = wines_df[col].apply(clean_and_convert)

wines_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   wine_link   2026 non-null   object 
 1   name        2023 non-null   object 
 2   year        2023 non-null   object 
 3   winery      2026 non-null   object 
 4   rating      2026 non-null   float64
 5   rating_qty  2026 non-null   object 
 6   price       1601 non-null   float64
 7   body        1974 non-null   float64
 8   tannis      1663 non-null   float64
 9   sweetness   1945 non-null   float64
 10  acidity     1974 non-null   float64
 11  notes       2026 non-null   object 
 12  pairings    2026 non-null   object 
 13  grapes      2026 non-null   object 
 14  region      2026 non-null   object 
 15  style       2021 non-null   object 
 16  image       2026 non-null   object 
dtypes: float64(6), object(11)
memory usage: 269.2+ KB


In [13]:
wines_df

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,image
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020,Luigi Bosca,4.8,575,164.99,73.43,50.90,13.61,44.74,"{'black fruit': 14, 'oaky': 13, 'earthy': 5, '...","['beef', 'lamb', 'poultry', 'game (deer, venis...","['Cabernet Sauvignon', 'Malbec']","['Argentina', 'Mendoza']",Argentinian Cabernet Sauvignon - Malbec,https://images.vivino.com/thumbs/_Bf6JTwYRpSX6...
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015,Catena Zapata,4.7,295,675.00,74.09,55.86,14.32,54.65,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017,Catena Zapata,4.7,218,580.00,74.09,55.86,14.32,54.65,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
3,https://www.vivino.com/US/en/monteviejo-la-vio...,La Violeta,2013,Monteviejo,4.7,202,150.00,73.06,40.91,21.48,36.61,"{'oaky': 51, 'black fruit': 49, 'earthy': 26, ...","['beef', 'lamb']",['100%Malbec'],"['Argentina', 'Mendoza', 'Uco Valley']",Argentinian Uco Valley Malbec Red,https://images.vivino.com/thumbs/FO-x9h3mQHSx9...
4,https://www.vivino.com/US/en/vina-cobos-cobos-...,Cobos Volturno,2018,Viña Cobos,4.7,198,399.99,71.05,45.63,24.25,38.45,"{'oaky': 86, 'black fruit': 49, 'spices': 17, ...","['beef', 'lamb', 'poultry', 'game (deer, venis...",['100%Cabernet Sauvignon'],"['Argentina', 'Mendoza', 'Perdriel']",Argentinian Cabernet Sauvignon - Malbec,https://images.vivino.com/thumbs/D1Mf1fYnRnage...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,https://www.vivino.com/etchart-cafayate-terroi...,Cafayate Terroir de Altura Torrontés,2021,Bodegas Etchart,3.9,110,NaN,47.89,NaN,19.51,56.47,"{'tree fruit': 3, 'red fruit': 1, 'floral': 1}","['vegetarian', 'spicy food', 'aperitif']",['Torrontés'],"['Argentina', 'Salta', 'Calchaqui Valley', 'Ca...",Argentinian Torrontés,https://images.vivino.com/thumbs/zbf30yF2Q_6j_...
2022,https://www.vivino.com/siesta-cabernet-sauvign...,Cabernet Sauvignon,2011,Siesta,3.9,108,NaN,65.15,54.17,13.75,41.57,"{'oaky': 22, 'black fruit': 14, 'red fruit': 1...","['beef', 'lamb', 'poultry']",['100%Cabernet Sauvignon'],"['Argentina', 'Mendoza']",Argentinian Cabernet Sauvignon,https://images.vivino.com/thumbs/hz2-G9-7TyGMC...
2023,https://www.vivino.com/gen-del-alma-jijiji-mal...,JIJIJI Malbec - Pinot Noir,2022,Gen del Alma,3.9,105,22.66,63.11,45.07,21.18,58.78,"{'red fruit': 60, 'black fruit': 27, 'earthy':...","['beef', 'lamb', 'pork', 'poultry', 'mushrooms...","['Malbec', 'Pinot Noir']","['Argentina', 'Mendoza', 'Uco Valley', 'Tupung...",Argentinian Malbec Red Blend,https://images.vivino.com/thumbs/hqHY7FTaRoucR...
2024,https://www.vivino.com/terrazas-de-los-andes-h...,High Altitude Vineyards Malbec,2021,Terrazas de los Andes,3.9,103,NaN,62.64,31.78,18.47,37.58,"{'oaky': 1756, 'black fruit': 1668, 'earthy': ...","['beef', 'lamb']",['100%Malbec'],"['Argentina', 'Mendoza', 'Lujan de Cuyo']",Argentinian Lujan de Cuyo Malbec Red,https://images.vivino.com/thumbs/xaqIkl7ERfa2e...


In [ ]:
#grapes En este caso el problema es que algunas uvas tienen pegado el porcentaje, ni siquiera hay un espacio, voy a revisar bien como trabajar en este caso

def extract_unique_grapes(df):
    unique_words = set()
    for entry in df['grapes'].dropna():
        matches = re.findall(r"'(.*?)'", entry)
        unique_words.update(matches)
    return list(unique_words)

# Llamar a la función con el DataFrame wines_df
unique_grapes = extract_unique_grapes(wines_df)

# Mostrar el resultado
print(unique_grapes)


['3%Petit Verdot', '21%Bonarda', 'Petit Verdot', 'Sémillon', 'Bonarda', 'Malbec', '70%Malbec', '100%Torrontés', '5%Tannat', 'Viognier', '100%Sémillon', '20%Petit Verdot', '50%Cabernet Sauvignon', '60%Malbec', 'Pinot Gris', '28%Cabernet Sauvignon', '69%Malbec', '12%Shiraz/Syrah', '29%Cabernet Sauvignon', '25%Malbec', '20%Cabernet Franc', '31%Cabernet Sauvignon', '90%Torrontés', '2%Petit Verdot', '15%Merlot', '35%Malbec', 'Béquignol Noir', '90%Cabernet Sauvignon', '65%Cabernet Sauvignon', '11%Cabernet Sauvignon', '2%Cabernet Franc', '67%Cabernet Sauvignon', 'Pinot Noir', '75%Cabernet Sauvignon', 'Moscatel', '100%Shiraz/Syrah', 'Pedro Ximenez', '100%Pinot Gris', 'Criolla Grande', '10%Bonarda', '91%Cabernet Sauvignon', 'Mencia', 'Garnacha', 'Sangiovese', '85%Cabernet Franc', 'Chenin Blanc', 'Grenache', 'Albariño', '15%Malbec', '14%Petit Verdot', '19%Petit Verdot', '8%Bonarda', '85%Malbec', 'Chardonnay', '25%Cabernet Sauvignon', '5%Shiraz/Syrah', '100%Verdejo', '100%Sauvignon Blanc', 'Argen

In [13]:
#region
def extract_unique_region(df):
    unique_words = set()
    for entry in df['region'].dropna():
        matches = re.findall(r"'(.*?)'", entry)
        unique_words.update(matches)
    return list(unique_words)

# Llamar a la función con el DataFrame wines_df
unique_region = extract_unique_region(wines_df)

# Mostrar el resultado
print(unique_region)

['Brazil', 'Rio Negro', 'Paraje Altamira', '17.5%', 'Argentina', 'Maipu', 'Mendoza', 'Rio Grande do Sul', 'La Consulta', 'Famatina', 'Tulum Valley', 'Uco Valley', 'Pedernal Valley', 'Vista Flores', 'Lujan de Cuyo', 'Calchaqui Valley', 'Campanha', 'Tupungato', 'Argentinian White', 'Agrelo', 'Perdriel', 'Serra Gaúcha', 'Tunuyán', 'San Rafael', '14%', 'Patagonia', 'San Juan', 'Cafayate Valley', 'Lunlunta', 'Argentinian Rosé', 'Salta', 'La Rioja', 'Gualtallary', '15%', 'Vale dos Vinhedos', 'San Carlos', 'Contains sulfites', 'Las Compuertas', 'Argentinian Mendoza Sparkling']


In [ ]:
# Extraer todas las palabras únicas de la columna 'notes'
def extract_unique_words(df):
    unique_words = set()
    for entry in df['notes'].dropna():
        try:
            notes_dict = ast.literal_eval(entry)  # Convertir la cadena en diccionario
            unique_words.update(notes_dict.keys())  # Agregar palabras únicas
        except (SyntaxError, ValueError):
            continue
    return list(unique_words)

# Crear un nuevo DataFrame con las palabras como columnas
def create_notes_dataframe(df):
    unique_words = extract_unique_words(df)
    
    # Crear DataFrame con las columnas deseadas
    new_df = df[['name', 'year', 'winery']].copy()

    # Agregar columnas para cada palabra y llenarlas con valores de 'notes'
    for word in unique_words:
        new_df[word] = df['notes'].apply(lambda x: ast.literal_eval(x).get(word, None) if pd.notna(x) else None)

    return new_df

# Aplicar la función al DataFrame
wines_notes_df = create_notes_dataframe(wines_df)
wines_notes_df.fillna(0, inplace=True)

wines_notes_df

,name,year,winery,dried fruit,red fruit,oaky,yeasty,spices,tropical,vegetal,black fruit,floral,ageing,citrus,tree fruit,earthy
0,Paraiso,2020,Luigi Bosca,0.0,2.0,13.0,0.0,4.0,0.0,0.0,14.0,2.0,0.0,0.0,0.0,5.0
1,Estiba Reservada,2015,Catena Zapata,3.0,21.0,109.0,2.0,25.0,0.0,1.0,50.0,6.0,6.0,2.0,0.0,24.0
2,Estiba Reservada,2017,Catena Zapata,3.0,21.0,109.0,2.0,25.0,0.0,1.0,50.0,6.0,6.0,2.0,0.0,24.0
3,La Violeta,2013,Monteviejo,1.0,11.0,51.0,2.0,11.0,0.0,1.0,49.0,11.0,1.0,0.0,0.0,26.0
4,Cobos Volturno,2018,Viña Cobos,2.0,15.0,86.0,1.0,17.0,0.0,0.0,49.0,2.0,5.0,2.0,0.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,Cafayate Terroir de Altura Torrontés,2021,Bodegas Etchart,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0
2022,Cabernet Sauvignon,2011,Siesta,3.0,14.0,22.0,1.0,8.0,0.0,1.0,14.0,1.0,1.0,0.0,0.0,9.0
2023,JIJIJI Malbec - Pinot Noir,2022,Gen del Alma,1.0,60.0,19.0,2.0,14.0,0.0,5.0,27.0,6.0,1.0,3.0,0.0,25.0
2024,High Altitude Vineyards Malbec,2021,Terrazas de los Andes,80.0,566.0,1756.0,62.0,354.0,9.0,40.0,1668.0,67.0,40.0,29.0,18.0,900.0


In [20]:
# Reescribi el codigo anterior pero dando un valor a cada nota de cero a 100 (fijate cual te gusta más)
def extract_unique_words(df):
    unique_words = set()
    for entry in df['notes'].dropna():
        try:
            notes_dict = ast.literal_eval(entry)
            unique_words.update(notes_dict.keys())
        except (SyntaxError, ValueError):
            continue
    return list(unique_words)

def create_normalized_notes_df(df):
    unique_words = extract_unique_words(df)

 
    new_df = df[['name', 'year', 'winery']].copy()
    for word in unique_words:
        new_df[word] = df['notes'].apply(
            lambda x: ast.literal_eval(x).get(word, 0) if pd.notna(x) else 0
        )

    note_cols = unique_words
    note_sums = new_df[note_cols].sum(axis=1)

    new_df[note_cols] = (new_df[note_cols].div(note_sums, axis=0) * 100).fillna(0).round(1)
    return new_df
wines_notes_normalized_df = create_normalized_notes_df(wines_df)

wines_notes_normalized_df


,name,year,winery,dried fruit,red fruit,oaky,yeasty,spices,tropical,vegetal,black fruit,floral,ageing,citrus,tree fruit,earthy
0,Paraiso,2020,Luigi Bosca,0.0,5.0,32.5,0.0,10.0,0.0,0.0,35.0,5.0,0.0,0.0,0.0,12.5
1,Estiba Reservada,2015,Catena Zapata,1.2,8.4,43.8,0.8,10.0,0.0,0.4,20.1,2.4,2.4,0.8,0.0,9.6
2,Estiba Reservada,2017,Catena Zapata,1.2,8.4,43.8,0.8,10.0,0.0,0.4,20.1,2.4,2.4,0.8,0.0,9.6
3,La Violeta,2013,Monteviejo,0.6,6.7,31.1,1.2,6.7,0.0,0.6,29.9,6.7,0.6,0.0,0.0,15.9
4,Cobos Volturno,2018,Viña Cobos,1.0,7.7,44.3,0.5,8.8,0.0,0.0,25.3,1.0,2.6,1.0,0.0,7.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,Cafayate Terroir de Altura Torrontés,2021,Bodegas Etchart,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,60.0,0.0
2022,Cabernet Sauvignon,2011,Siesta,4.1,18.9,29.7,1.4,10.8,0.0,1.4,18.9,1.4,1.4,0.0,0.0,12.2
2023,JIJIJI Malbec - Pinot Noir,2022,Gen del Alma,0.6,36.8,11.7,1.2,8.6,0.0,3.1,16.6,3.7,0.6,1.8,0.0,15.3
2024,High Altitude Vineyards Malbec,2021,Terrazas de los Andes,1.4,10.1,31.4,1.1,6.3,0.2,0.7,29.8,1.2,0.7,0.5,0.3,16.1


In [ ]:
# Extraer todas las palabras únicas de la columna 'pairings', crear df, usar 0 y 1 para los elementos que esten o no en cada registro
def extract_unique_pairings(df):
    unique_pairings = set()
    for entry in df['pairings'].dropna():
        try:
            pairings_list = ast.literal_eval(entry)
            unique_pairings.update(pairings_list)
        except (SyntaxError, ValueError):
            continue
    return list(unique_pairings)

def create_pairings_df(df):
    unique_pairings = extract_unique_pairings(df)

    
    new_df = df[['name', 'year', 'winery']].copy()

   
    for pairing in unique_pairings:
        new_df[pairing] = df['pairings'].apply(
            lambda x: 1 if pd.notna(x) and pairing in ast.literal_eval(x) else 0
        )

    return new_df


wines_pairings_df = create_pairings_df(wines_df)

# Mostrar resultado
wines_pairings_df


,name,year,winery,pork,goat's milk cheese,poultry,vegetarian,veal,any junk food will do,beef,...,spicy food,aperitif,mature and hard cheese,cured meat,lean fish,lamb,pasta,mild and soft cheese,mushrooms,"rich fish (salmon, tuna etc)"
0,Paraiso,2020,Luigi Bosca,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,Estiba Reservada,2015,Catena Zapata,0,0,1,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
2,Estiba Reservada,2017,Catena Zapata,0,0,1,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
3,La Violeta,2013,Monteviejo,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,Cobos Volturno,2018,Viña Cobos,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,Cafayate Terroir de Altura Torrontés,2021,Bodegas Etchart,0,0,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
2022,Cabernet Sauvignon,2011,Siesta,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2023,JIJIJI Malbec - Pinot Noir,2022,Gen del Alma,1,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
2024,High Altitude Vineyards Malbec,2021,Terrazas de los Andes,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
